# Import libraries

In [ ]:
# Upload File using ipyfilechooser library
from ipyfilechooser import FileChooser
from ipywidgets import interact, interactive, fixed, interact_manual
from tqdm.notebook import tqdm, trange
# Video Player
from IPython.display import Video
import time 
# Get the root directory of the project
from pyprojroot import here
# Copy File
import shutil
# Widget Packages
import ipywidgets as widgets
# In case widget extension not working
# jupyter nbextension enable --py widgetsnbextension
import os
import sys

# Data Exploration Section
R2 (Epic): As a user, I want a "Data Exploration" section in the notebook so that I can load and display video data from the TSU project.

## Video Upload / Choose using ipyfilechooser
R2 (Story): As a user, I want to upload/choose files from the data folder through an appropriate UI component (E.g. Browse files) in a notebook code cell so that I can pick and choose the video data I would like to process

In [ ]:
def videoselectorinput():
    starting_directory = './data/video'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser
def videoselectoroutput():
    starting_directory = './pipeline/video/output'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser

In [ ]:
video = videoselectorinput()

## Upload selected video to the data folder (If needed)

In [ ]:
# Upload Function
# from pyprojroot import here
# import shutil
def upload(video):
    print(video.selected)
    source = video.selected
    # Source path
    # Destination path
    destination = (here("./data/video"))

    # Copy file from the selected path
    try:
        shutil.copy(source, destination)
        print("File copied successfully.")

    # If source and destination are same
    except shutil.SameFileError:
        print("Source and destination represents the same file.")

    # If destination is a directory.
    except IsADirectoryError:
        print("Destination is a directory.")

    # If there is any permission issue
    except PermissionError:
        print("Permission denied.")

    # For other errors
    except:
        print("Error occurred while copying file.")

upload(video)

## Video Playback
R2 (Story): As a user, I want to see video playback of the chosen video file in an output cell so that I can check if it is the right video data I would like to process

In [ ]:
# Select Video
video = videoselectorinput()

In [ ]:
print(video.selected)
print(video.selected_filename)
print(video.selected_path)

In [ ]:
full_path = video.selected
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

# Inference Section

R3 (Epic): As a user, I want to have an "Inference" section in the notebook so that I can perform inference using a pretrained HOI ML model based on the TSU project.

## Load a pretrain model
R3 (Story): As a user, I want to load a pre-trained model using an appropriate UI component so that I can easily load the model.

In [ ]:
# Select model
path =here()
os.chdir(path)
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./pipeline/models"): 
    modelList += [x]

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

## Choose Input video to load into TSU Project
R3 (Story): As a user, I want to choose an  input video files and other related input files, using an appropriate UI component, from the TSU project so that the system is able to pass the right files to the model.

In [ ]:
# Select Video
video = videoselectorinput()

## Run the model 
R3 (Story): As a user, I want to see inference results in the form of an output video with captions that indicate the current detected activity in each video frame so that I am able to see the inference results clearly on the screen.

In [ ]:
path =here("./pipeline")
%cd $path
model = modelDropdown.value
loadmodel = './models/' + model
videoPath = video.selected
videoFile = video.selected_filename
print(videoFile)
print(videoPath)
%run -it test.py  -input_video_file $videoFile -model $model  -load_mode $loadmodel -video_path $videoPath

## Output Video to view the inference result 

In [ ]:
videoFileName = video.selected_filename[:-4]
full_path = (os.path.join(here("./pipeline/video/output/"),f"{videoFileName}_caption.mp4"))
print(full_path)
Video(full_path, embed=True, width=540, html_attributes="controls muted autoplay")

# Training Section
R4 (Epic): As a user, I want to create a "Training" section in the netbook so that I can train a HOI ML model based on the TSU project.

## Run_PDAN.sh for training
R4 (Story): As a user, I want to able to change the value for the argument in run_PDAN shell script with a UI so that I does not need to keep changing the value directly in the shell script.

In [ ]:
root_path=here()
os.chdir(root_path)
title = widgets.Label("Run_PDAN")
style = {'description_width': '90px'}

dataset_input = widgets.Text(
    value='TSU',
    placeholder='Enter Dataset Name',
    description='Dataset:',
    disabled=False,
    style=style
)

mode_input = widgets.Text(
    value='rgb',
    placeholder='Enter Mode',
    description='Mode:',
    disabled=False,
    style=style
)

split_input = widgets.Text(
    value='CS',
    placeholder='Enter Split Setting',
    description='Split Setting:',
    disabled=False,
    style=style
)

# Using Dropdown method
model_list = []
directoryModel = "./pipeline/models" # Directory of the dataset (NPY files)

# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryModel):
    if os.path.isdir(os.path.join(directoryModel, x)) == False:
        model_list.append(x)
        
model_input = widgets.Dropdown(
    options=model_list,
    value=model_list[0],
    description='Model:',
    style=style
)


train_input = widgets.Checkbox(
    value=True,
    description = 'Train',
    disabled=False,
    indent=True,
    style=style
)

num_channel_input =  widgets.BoundedIntText(
    value=512,
    min=1,
    max=1000,
    step=1,
    description='Num Channel:',
    disabled=False,
    style=style
)

lr_input = widgets.FloatText(
    value=0.0002,
    description='Learning Rate:',
    disabled=False,
    style=style
)

kernel_size_input =  widgets.BoundedIntText(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Kernel Size:',
    disabled=False,
    style=style
)

aptype_input = widgets.Text(
    value='map',
    placeholder='Enter APType',
    description='APType:',
    disabled=False,
    style=style
)

epoch_input = widgets.BoundedIntText(
    value=140,
    min=1,
    max=1000,
    step=1,
    description='Epoch:',
    disabled=False,
    style=style
)

batch_size_input = widgets.Dropdown(
    options=['1', '2', '4', '8', '16', '32', '64', '128', '256', '512', '1024'],
    value='1',
    description='Batch_Size:',
    disabled=False,
    style=style
)


comp_info_input = widgets.Text(
    value='TSU_CS_RGB_PDAN',
    placeholder='Enter Compute Info',
    description='Compute Info:',
    disabled=False,
    style=style
)

button = widgets.Button(
    description='Save',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save Pref',
    icon='check', # (FontAwesome names without the `fa-` prefix)
    style=style
)

def selectWidgetSet(b):
    print("Selected Dataset: " , dataset_input.value)
    print("Selected Mode: " , mode_input.value)
    print("Selected Split Setting: " , split_input.value)
    print("Selected Model: " , model_input.value)
    print("Selected Train: " , train_input.value)
    print("Selected Num Channel: " , num_channel_input.value)
    print("Selected Learning Rate: " , lr_input.value)
    print("Selected Kernel Size: " , kernel_size_input.value)
    print("Selected APType: ", aptype_input.value)
    print("Selected Epoch: ", epoch_input.value)
    print("Selected Batch: " , batch_size_input.value)
    print("Selected Compute Info: " , comp_info_input.value)
    path =here("./pipeline")
    %cd $path
    %run -i train.py -dataset $dataset_input.value -mode $mode_input.value -split_setting $split_input.value -model $model_input.value -train $train_input.value -num_channel $num_channel_input.value -lr $lr_input.value -kernelsize $kernel_size_input.value -APtype $aptype_input.value -epoch $epoch_input.value -batch_size $batch_size_input.value -comp_info $comp_info_input.value
display(title, dataset_input, mode_input,split_input, model_input , train_input, num_channel_input, lr_input, kernel_size_input, aptype_input, epoch_input, batch_size_input, comp_info_input ,button)
button.on_click(selectWidgetSet)

## Choose dataset folder to use for training

R4 (Story): As a user, I can choose a dataset subfolder, using appropriate UI elements, from the data folder to use for the training so that I can select the data for training.

In [ ]:
# Using Dropdown method
dataset_list = []
directoryDataset = "./data/dataset/" # Directory of the dataset (NPY files)

# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    value=dataset_list[0],
    description='Dataset:')

# Widgets
datasetConfirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

# Function on what happen when confirm is been click.
def selectDataSet(b):
    print("Selected Dataset: " , datasetDropDown.value)
   
datasetConfirmButton.on_click(selectDataSet)
datasetBox = widgets.VBox([widgets.HBox([datasetDropDown, datasetConfirmButton])])
datasetBox

In [ ]:
# Using Browse folder method
def folderSelector():
    starting_directory = './data/dataset'
    chooser = FileChooser(starting_directory)
    chooser.show_only_dirs = True
    display(chooser)
    return chooser

datafolder = folderSelector()

R4 (Story): As a user, I can add the trained model to the list of pre-trained models that can be chosen in R3 after its training so that I can add my trained model to pre-trained models

In [ ]:
change_list = ['Add', 'Remove']

confirmChangeButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)

modelChangeDropDown = widgets.Dropdown(
    options=change_list,
    description='Add/Remove: ')

def selectChangeSet(b):
    global choice
    choice = modelChangeDropDown.value
    print("Selected: " , modelChangeDropDown.value)
    
confirmChangeButton.on_click(selectChangeSet)
display(modelChangeDropDown, confirmChangeButton)

In [ ]:
def addModel():
    starting_directory = '.'
    chooser = FileChooser(starting_directory)
    display(chooser)
    return chooser

#pretrained_models = ['TSU_', 'NVIDIA Setp Model']
pretrained_models = []



In [ ]:
def selected_file(add_model):
    if add_model.selected_filename == None:
        return  ''
    else:
        from pathlib import Path
        file_name = Path(add_model.selected_filename).stem
        return file_name
    
file_name = selected_file (add_model)

#models_list = ['TSU', 'NVIDIA Set Model']
models_list = []

def checkExist(filename):
    with open('models_list.txt', 'r') as f:
        if filename in f.read():
            f.close()
            return True
        else:
            f.close()
            return False
        
def addToList(filename):
    if checkExist(filename):
        print(filename, " already exist")
    else:
         with open('models_list.txt', 'a') as f:
            f.write(filename + "\n")
            
def removeFromList(filename):
    if checkExist(filename):
        with open('models_list.txt', 'r') as file:
            text = file.read()


        # Delete text and Write
        with open('models_list.txt', 'w') as file:
            # Delete
            new_text = text.replace(filename, '')
            # Write
            file.write(new_text)
    else:
        print(filename, " does not exist")
    
    
if choice == 'Add':
    addToList(file_name)
elif choice == 'Remove':
    removeFromList(file_name)
else:
    print('Invalid choice')

# Testing Section
R5 (Epic): As a user, I want to have a "Testing" section in the notebook so that I can evaluate a trained model based on the TSU project.

R5 (Story): As a user, I want to load a pretrained model using an appropriate UI component so that I can easily choose the type of pretrained model I would like to process the data with.

In [ ]:
# Select model
path =here()
os.chdir(path)
modelList = [] 

# Select from the list of model in the pipeline/models folder
for x in os.listdir("./pipeline/models"): 
    modelList += [x]

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
modelDropdown = widgets.Dropdown(
    options=modelList,
    value=modelList[0],
    description='Model:')
# Function on what happen when confirm is been click.
def selectWidgetSet(b):
    print("Selected: " , modelDropdown.value)

confirmButton.on_click(selectWidgetSet)
modelBox = widgets.VBox([widgets.HBox([modelDropdown, confirmButton])])
modelBox

# Testing Section
R5 (Epic): As a user, I want to have a "Testing" section in the notebook so that I can evaluate a trained model based on the TSU project.

# NVIDIA STEP Section
R6 (Epic): As a user, I want to able to configure the notebook using appropriate UI elements coupled with the right .py modules so that R2-5 can be performed based on another pipeline, e.g., the NVIDIA STEP pipeline 

## Pipeline Selection
R6(Story): As a user, I want to create appropriate UI elements to allow for switching pipelines so that I can test the different models



In [ ]:
w = widgets.Dropdown(
    options=['TSU', 'NVIDIA SMarthome'],
    value='TSU',
    description='Pipeline:',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

w.observe(on_change)

display(w)

## Nvidia Step Pipeline
R6(Story): As a user, I want to ensure selected pipeline's dependencies are changed to ensure the right dependencies are given to the appropriate models so that the selected model will be run

In [ ]:
%cd NVIDIA-STEP-MODEL/STEP

In [ ]:
!ipython setup.py build develop

In [ ]:
!python demo.py

In [ ]:
from IPython.display import Image
Image(filename='datasets/demo/frames/results/2/frame0000.jpg') 

T05-119 As a user, I want to have multiple dropdowns to select the input (1) dataset (2) video so that I can select the dataset videos easily

In [ ]:
# Select video
video_list = [] 
dataset_list = [] 
directoryVideo = "./data/video/" # Directory of the video (.mp4)
directoryDataset = "./data/dataset/" # Directory of the dataset (NPY files)

# Store the folder in the  dataset into dataset list
for x in os.listdir(directoryDataset):
    if os.path.isdir(os.path.join(directoryDataset, x)):
        # print(os.path.join(directoryDataset, x))
        dataset_list.append(x)

# Store the video names into video list
for x in os.listdir(directoryVideo):
    if x.endswith(".mp4"):
        video_list.append(x)

datasetDropDown = widgets.Dropdown(
    options=dataset_list,
    description='Dataset:')

videoDropDown = widgets.Dropdown(
    options=video_list,
    description='Video: ')

# Widgets
confirmButton = widgets.Button(
    description='Confirm',
    disabled=False,
    button_style='success',
    icon='check'
)
# Function on what happen when confirm is been click.
def selectTrimSet(b):
    print("Selected Dataset: " , datasetDropDown.value)
    print("Selected Video: " , videoDropDown.value)

confirmButton.on_click(selectTrimSet)
display(datasetDropDown, videoDropDown, confirmButton)

T05-128: As a User, I want pre-process the input video into frames for Nvidia STEP model, So that I can run the model successfully.

In [ ]:
!pip3 install opencv-python

In [ ]:
import cv2 as cv

cap= cv.VideoCapture('Input/sample.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv.imwrite('datasets/demo/frames/3/output'+str(i)+'.jpg',frame)
    i+=1

cap.release()
cv.destroyAllWindows()

T05-127 As a User, I want the output of Nvidia STEP model to be in a video format, So that I can easily view the results of the model.

In [ ]:
!pip3 install opencv-python

In [ ]:
!pip3 install os-sys

In [ ]:
import cv2
import numpy as np
import glob

img_array = []
for filename in glob.glob('datasets/demo/frames/1/*.jpg'):
    img = cv2.imread(filename) 
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)


out = cv2.VideoWriter('Output/project.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()